In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""         
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_productivity_sft

--PARTITION BY DATE(START_TIME) 

TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)

AS
WITH ranked_data AS (
  SELECT
    CLIENT_CODE,
    WAREHOUSE_CODE,
    WAREHOUSE_SITE_ID,
    OBJECT_CODE,
    REF_CODE,
    JOB_TYPE,
    IS_EVEN,
    EMPLOYEE_ID,
    EMPLOYEE_NAME,
    START_TIME,
    END_TIME,
    TOTAL_SKU,
    TOTAL_SKU_PROCESSED,
    TOTAL_UNIT,
    TOTAL_UNIT_PROCESSED,
    ROW_NUMBER() OVER(PARTITION BY OBJECT_CODE, JOB_TYPE, REF_CODE, EMPLOYEE_ID ORDER BY PROC_DATE DESC) AS row_num
  FROM {catalog_name}.udp_wcm_silver_eventhub.productivity_sft
)
SELECT
  CLIENT_CODE,
  WAREHOUSE_CODE,
  WAREHOUSE_SITE_ID,
  OBJECT_CODE,
  REF_CODE,
  JOB_TYPE,
  IS_EVEN,
  EMPLOYEE_ID,
  EMPLOYEE_NAME,
  START_TIME,
  END_TIME,
  TOTAL_SKU,
  TOTAL_SKU_PROCESSED,
  TOTAL_UNIT,
  TOTAL_UNIT_PROCESSED
FROM ranked_data
WHERE row_num = 1
""")